# Ingestão Bronze - Cadastro de Cliente

Este notebook realiza a leitura de um arquivo Excel contendo o cadastro de clientes
e grava os dados em formato Delta na camada **Bronze**.

**Origem dos dados:**
- Arquivo: `D:\Projetos\Jornada_financas_pessoais\data\source\cadastro_cliente.xlsx`

**Destino:**
- Caminho Delta: `D:\Projetos\Jornada_financas_pessoais\data\delta\bronze\raw_cadcliente`
- Tabela Delta: `bronze.raw_cadcliente`


## Imports

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession

## Start Spark Session

In [ ]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura de Arquivo Formato Excel") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")

## Variables

In [ ]:
# Caminhos e variáveis
excel_path = "D:/Projetos/Jornada_financas_pessoais/data/source/cadastro_cliente.xlsx"
delta_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cadcliente"
table_name = "bronze.raw_cadcliente"


# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"

## Leitura do arquivo Excel

In [ ]:
# Leitura do Excel (aba única)
df_cadcliente = (
    spark.read.format("com.crealytics.spark.excel")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(excel_path)
)

# Exibe o schema e algumas linhas
df_cadcliente.printSchema()
display(df_cadcliente.limit(10))

## Escrita em formato Delta (Camada Bronze RAW)

In [ ]:
# Grava em formato Delta
(
    df_cadcliente.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(delta_path)
)

## Registro da tabela Delta no catálogo

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS bronze COMMENT 'Camada Bronze do Data Lake - Dados brutos'")
spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING DELTA LOCATION '{delta_path}'")

## Validação da ingestão

In [ ]:
df_validacao = spark.read.format("delta").load(delta_path)
print(f"Total de registros: {df_validacao.count()}")
display(df_validacao.limit(10))